Date: 29 Apr 2018

Note on `pymongo`
---
## 0. Introduction
---

### 0.1 Import library

In [9]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

### 0.2 Make connection to the server

In [3]:
# Explicitly specify host(str) and port(int).
#Client = MongoClient('localhost',27017)

# Using URI to specifiy host and port
#Client = MongoClient('mongodb://localhost:27017/')

# Using default host(localhost) and post(27017).
client = MongoClient()

### 0.3 Basic structure

In [4]:
db = client.fit5148_db
type(db)

pymongo.database.Database

In [5]:
type(db.FIT)

pymongo.collection.Collection

In [6]:
type(db.FIT.find({}))

pymongo.cursor.Cursor

In [7]:
type(db.FIT.find()[0])

dict

After querying the database, the query results will be returned in a `Cursor` type, data within can be retrieved by **iterator**:

In [10]:
for data in db.FIT.find():
    pprint(data)

{'_id': 126,
 'course': 'MBIS',
 'name': {'first': 'Peter', 'last': 'Chen'},
 'result': [{'mark': 99,
             'semester': 1,
             'unit_code': 'FIT9132',
             'unit_name': 'Database',
             'year': 2017}]}
{'_id': 124,
 'course': 'MBIS',
 'name': {'first': 'Albert', 'last': 'Einstein'},
 'result': [{'mark': 90,
             'semester': 1,
             'unit_code': 'FIT9131',
             'unit_name': 'Programming',
             'year': 2017},
            {'mark': 90,
             'semester': 1,
             'unit_code': 'FIT9132',
             'unit_name': 'Database',
             'year': 2017}]}
{'_id': 127,
 'course': 'MIT',
 'name': {'first': 'Charles', 'last': 'Babbage'},
 'result': [{'mark': 95,
             'semester': 1,
             'unit_code': 'FIT9132',
             'unit_name': 'Database',
             'year': 2017},
            {'semester': 2,
             'unit_code': 'FIT9131',
             'unit_name': 'Programming',
             'year': 2017

Or, using **indexing** to retrieve data:

In [6]:
pprint(db.FIT.find()[0])

{'_id': 126,
 'course': 'MBIS',
 'name': {'first': 'Peter', 'last': 'Chen'},
 'result': [{'mark': 99,
             'semester': 1,
             'unit_code': 'FIT9132',
             'unit_name': 'Database',
             'year': 2017}]}


In [11]:
sample = db.sample

## Insert
1. Insert one document: `db.collection.insert_one()`
2. Insert mutiple document: `db.collection.insert_many()`  
**Note**: In Pymongo, documents are stored in dictionary type. Thus, only `dict` type objects are valid input when using `insert_one()`. On the other hand, a list of `dict` objects is allowed as input in `insert_many()`.

In [13]:
evals = [{"results":[{"item": "content","score": 9},{"item": "presentation","score": 6}]},
        {"results":[{"item": "content","score": 8},{"item": "presentation","score": 8}]}]

result = db.sample.insert_many(evals)

After insert multiple documents, we can retrieve the `inserted_ids`

In [14]:
result.inserted_ids

[ObjectId('5afa5a3659525d03002e53dc'), ObjectId('5afa5a3659525d03002e53dd')]

## Update

`update_one` and `update_many()`  
**Cannot update `_id` field**  
`result.matched_count`: print out how many documents were updated  
`replace_one()`

## Delete

`delete_one()` and `delete_many()`  
**take one condition to match**  
`result.deleted_count` to check whether document(s) have been deleted.  
### Drop
`collectionName.drop()`: To delete all documents from a collection  
`collectionName.rename('newName')`: change the name of a collection  

## Read

* `find()`  
* `find_one()`
* `collectionName.count()` or `collectionName.find({criteria}).count()`

### Range
* `collectionname.find({"field":{"$lte" : 50 }})`  
**More details of operators find mongo shell Note**

### Multiple conditions
* `collectionName.find({"field_1":condition1, "field2":{"$operator": condition_2})`

### Logical Conjustion
* `$or`
* `$and`

Example:  
```
collection.find(
    {"$or":
        [{"field1": condition1},
         {"field2":{
             "$operator": condition2
             }
         }
        ]
    }
)
```

## Sort

`pymongo.ASCENDING` and `pymongo.DESCENDING`

Example:  
* `collection,find({criteria}).sort("field_1", pymongo.ASCENDING)`
* `collection,find({criteria}).sort("field_1", pymongo.DESCENDING)`
* `collection,find({criteria}).sort([(option_1), (option_2)])`

## Indexing

Create a unique index on key, for example, `foo.something`:  
* `collectionName.create_index([('foo.something', pymongo.ASCENDING)], unique=True)`

**Note**: Now we have 2 indexes:
1. index on `_id`
2. index on `foo.something`

In [27]:
c_sample = db.sample
sample.create_index([('results.score', pymongo.ASCENDING)], unique=True)
sample.create_index([('results.score', pymongo.DESCENDING)], unique=True)

'results.score_-1'

To check what field have indexes: 
1. `-1` means descending order index
2. `1` means ascending oreder index

In [28]:
sorted(list(sample.index_information()))

['_id_', 'results.score_-1', 'results.score_1']

## Aggregate

### Count the number of record matched the criteria

```
collectionName.aggregate(
    ["$group":{
        "_id":"$field", 
        "count":{"$sum":1}
        }]
    )
```

### Calculation the average value of field

```
collectionName.aggregate(
    ["$group":{
        "_id":"$field", 
        "Avg":{"$avg":"$field"},
        "count":{"$sum":1}
        }]
    )
```

## Join

Example collections:

In [29]:
users = db.users
units = db.units

Take a look at these collection:

In [33]:
pprint(users.find()[0])

{'_id': ObjectId('5af1244ad87cca47ac32bb0a'),
 'name': {'first': 'Marie', 'last': 'Currie'},
 'sid': 123}


In [34]:
pprint(units.find()[0])

{'_id': ObjectId('5af124edd87cca47ac32bb58'),
 'mark': 100,
 'semester': 2,
 'sid': 124,
 'unit_code': 'FIT9132',
 'unit_name': 'Database',
 'year': [2017]}


### Using `$lookup` with `aggregation()`

**users** inner join with **units** on `sid`

In [49]:
results1 = users.aggregate([{ 
    "$lookup":
    { "from": "units", 
      "localField": "sid", 
      "foreignField" : "sid", 
      "as": "completed_units" }
}])
for document in results1:
    pprint(document)

{'_id': ObjectId('5af1244ad87cca47ac32bb0a'),
 'completed_units': [{'_id': ObjectId('5af124edd87cca47ac32bb59'),
                      'mark': 80,
                      'semester': [1, 2],
                      'sid': 123,
                      'unit_code': 'FIT9131',
                      'unit_name': 'Programming',
                      'year': [2016]},
                     {'_id': ObjectId('5af124edd87cca47ac32bb5a'),
                      'mark': 100,
                      'semester': 1,
                      'sid': 123,
                      'unit_code': 'FIT9132',
                      'unit_name': 'Database',
                      'year': [2017]}],
 'name': {'first': 'Marie', 'last': 'Currie'},
 'sid': 123}
{'_id': ObjectId('5af1244ad87cca47ac32bb0b'),
 'completed_units': [{'_id': ObjectId('5af124edd87cca47ac32bb58'),
                      'mark': 100,
                      'semester': 2,
                      'sid': 124,
                      'unit_code': 'FIT9132',
           

In [47]:
results3 = units.aggregate([
    {"$unwind": "$sid"},
    {
    "$lookup":
    { "from": "users", 
      "localField": "sid", 
      "foreignField" : "sid", 
      "as": "completed_units" }},
    {"$project":{"_id":0, "sid":1, "year":1, "completed_units":1}}
])
for document in results3:
    pprint(document)

{'completed_units': [{'_id': ObjectId('5af1244ad87cca47ac32bb0b'),
                      'name': {'first': 'Albert', 'last': 'Einstein'},
                      'sid': 124}],
 'sid': 124,
 'year': [2017]}
{'completed_units': [{'_id': ObjectId('5af1244ad87cca47ac32bb0a'),
                      'name': {'first': 'Marie', 'last': 'Currie'},
                      'sid': 123}],
 'sid': 123,
 'year': [2016]}
{'completed_units': [{'_id': ObjectId('5af1244ad87cca47ac32bb0a'),
                      'name': {'first': 'Marie', 'last': 'Currie'},
                      'sid': 123}],
 'sid': 123,
 'year': [2017]}


<font color='red'>**`$unwind` operator should be always preceding to `$lookup`**</font>